<a href="https://colab.research.google.com/github/WRFitch/fyp/blob/main/src/fyp_ai_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AI data analysis

### Setup
- Install & import necessary libraries
- Mount drive
- Import and define handy variables 

In [ ]:
# Sometimes the colab fastai version can be wrong, so we reinstall with no cache
# reinstalling, and restarting runtime should fix any major issues, including 
# CUDA OOM error
!pip uninstall -y fastai
!pip install -U --no-cache-dir fastai

In [ ]:
import pandas as pd

from fastai.vision.all import *
from google.colab import drive

drive.mount('/content/drive')

In [ ]:
%rm -rf /content/fyp

In [ ]:
%cd /content
!git clone https://github.com/WRFitch/fyp.git

In [ ]:
# Import fyputil library
%cd /content/fyp/src/fyputil
import constants as c
import fyp_utils as fyputil
%cd /content

### Data Setup

In [3]:
ghg_df = pd.read_csv(c.ghg_csv)
ghg_df = fyputil.normGhgDfProperly(ghg_df)

In [4]:
def getGhgsAsArr(img_path):
  return fyputil.getGhgsAsArr(img_path, ghg_df)

def imgIsInDf(path):
  return fyputil.imgIsInDf(path, ghg_df)

def getGhgImgs(path):
  return get_image_files(path).filter(imgIsInDf)

In [5]:
# TODO implement multiple transforms pipeline
# TODO revisit image normalisation
# TODO ensure random data splitter is ok (it's not)

ghg_block = DataBlock(
    blocks = (ImageBlock, RegressionBlock),
    get_items = getGhgImgs,
    get_y = getGhgsAsArr,
    item_tfms = Resize(460), 
    batch_tfms = aug_transforms(size=224, max_warp=0.05, max_zoom=1.0, max_rotate=45),
    splitter  = RandomSplitter()
)

ghg_dl = ghg_block.dataloaders(c.big_png_dir)

In [9]:
aug_transforms??

In [ ]:
ghg_dl.show_batch(nrows=9, max_n=9, figsize = (50,50))

In [ ]:
ghg_block.summary(c.big_png_dir)

In [ ]:
bigimgs = get_image_files(c.big_png_dir)
len(bigimgs)

## Training

### Image Recognition and Feature Extraction. 

- Train image-based predictor to guess greenhouse gas concentrations based on 1km square of land. 
  - Transfer an ImageNet predictor to work top-down
  - Start by predicting one ghg and expand from there
- Use image predictor to extract a basic feature set by slicing the network at different points. The idea is to limit the amount of data going into the tabular recommender, while transferring as much useful data as possible. We want to implicitly extract GHG-emitting features of each image without losing any detail, as a form of convolutional preprocessing. 


In [ ]:
# TODO experiment with variable floating-point accuracy 
# TODO experiment with smaller networks
# TODO experiment with batch normalisation
# TODO experiment with adding a 2-layer head to the network to ensure decent conversions 
#learn = cnn_learner(ghg_dl, resnet152, y_range=(0, 100),  metrics=rmse).to_fp16()
learn = cnn_learner(ghg_dl, resnet152, y_range=(0, 100),  metrics=rmse)
name = "fresh learner"
learn.save(name)

In [ ]:
learn.load(name)

In [ ]:
# TODO examine 3d representation of problem space re: local optima 
learn.lr_find()

In [10]:
# TODO When cutting release branch, update these to be the actual learning rates used for training the final branch. 
# TODO experiment with different discriminative learning rates
lr = 0.033

In [113]:
learn.freeze()

In [ ]:
# Fit the first layer before unfreezing to get the network halfway there. If it overfits for now, that's not really a problem. 
learn.fit(1, lr) 

In [ ]:
# Saving mid-training, so I can figure out a decent training pathway
learn.save("mid-training")

In [ ]:
learn.load("mid-training")

In [28]:
learn.unfreeze()
# Unfreeze the rest of the layers. How much do the rest of these layers need training? 
# Make sure to find new LRs. 

In [25]:
lrs = slice(0.003, 0.1)
lrs2 = slice(3e-4, 1e-3)
lrs3 = slice(1e-6, 1e-3)

In [ ]:
# TODO test fit_one_cycle - this may be better! it also includes discriminative lr slicing
learn.fit_one_cycle(10, lr_max=lrs2)

In [ ]:
learn.save("fine-tuning")

In [ ]:
learn.load("fine-tuning")

In [ ]:
learn.save("xfine-tuning")

In [ ]:
learn.load("xfine-tuning")

## Evaluate Model Performance 

### Plot results 

In [ ]:
learn.validate()

In [ ]:
learn.show_results(ds_idx=4, dl=ghg_dl, nrows=9, max_n=9, figsize = (50,50))

### Plot model statistics 

#### Plot layer stats

This allows us to see what the mean std and pct activation levels are, letting us see areas of the network that require further analysis 

In [ ]:
learn.activation_stats.plot_layer_stats(151) 

In [ ]:
learn.recorder.plot_loss()

In [ ]:
learn.activation_stats.color_dim(-4)

### Export the model

#### Cutting a neural encoder
I hope this works

In [ ]:
encoder = create_body(resnet152, cut = -2)

#### Exporting main model


In [ ]:
# Export model so we can use it for other things. Note - this kills the model 
#TODO find better naming convention 
new_model = "080321_resnet152_7k-imgs_fit-one-cycle"
learn.export(f"{c.model_dir}/{new_model}.pkl")

In [ ]:
# Import model and test to see if it hasn't broken in the export process.
imported_learner = load_learner(f"{c.model_dir}/{c.model_name}.pkl")

In [ ]:
# Predict from imported learner
imported_learner.predict(f"{c.png_dir}/-0.73212695655741_51.2533785354393.png")

#### Notes on Image Predictions

A lower learning rate appears to cause slower training with more sophisticated conclusions. Sophistication also appears to arise from a deeper network, but I'm hitting a wall at roughly 0.6 rmse.

Effectively, this network recognises certain features of high-GHG land. Depending on sophistication, this may include airports, power plants, or other rare features, as well as recognising different types of wilderness or residential districts. This will be used to extract a feature set for a tabular recommender, which can then be used to find more accurate readings. 